|  Surname | Name   | Matricola   | Accademic Mail   |
|---|---|---|---|
|Rossetto   | Eric     | 982594    | eric.rossetto@studio.unibo.it |
|Wen          | Xiaowei   | 982501  | xiaowei.wen@studio.unibo.it  |  


MSc student in Artificial Intelligence
@ Alma Mater Studiorum, University of Bologna  
August, 2021

# Importing libraries

In [17]:
from z3 import *
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as mpatches
import time
import json

# Defining functions 

In [18]:
from tools import *

# Defining function for SMT instance

In [19]:
def vlsi_instance(W, H, dims, width, max_height, time_out = 300):
  """
    Variable declaration
    W: width of all circuits
    H: height of all circuits
    dims: number of circuits
    width: width of the board/ solution
    max_height: maximum height reachable by stacking all circuits. 
    time_out: timeout for the solver in seconds
  """
  # decision variable for the height
  all_constraints = []
  height = Int('height')

  rot = [Int(f"rot_{i}") for i in range(dims)]
  rotation_cons = [And(rot[i]>=0, rot[i]<=1) for i in range(dims)]
  all_constraints += rotation_cons

  # X component of the solution where to place the circuits
  X = [Int(f"X_{i}") for i in range(dims)]
  
  # Y component of the solution where to place the circuits
  Y = [Int(f"Y_{i}") for i in range(dims)]

  # width boundaries constraints
  boundaries_c_width = [And(0 <= X[i], X[i] +(1 - rot[i]) * W[i] +  rot[i]*H[i] <= width) for i in range(dims)]
  all_constraints+= boundaries_c_width

  # height boundaries constraints
  boundaries_c_height = [0 <= Y[i] for i in range(dims)] 
  all_constraints += boundaries_c_height
  
  # height constraints
  height_constraint = [Y[i] + (1 - rot[i]) * H[i] + rot[i] * W[i]  <= height for i in range(dims)]
  all_constraints += height_constraint

  biggest = int(np.argmax(np.array(H)*np.array(W)))
  # symmetry breaking 
  all_constraints +=  [And(X[biggest]==0 , Y[biggest]==0 )]
  # all_constraints +=  [And(X[biggest]*2<= height , Y[biggest]*2 <=width )]

  # height constraints
  height_constraint = [Y[i]+H[i]<=height for i in range(dims)]
  all_constraints += height_constraint

  # no overlapping constraints
  no_overlapping = []    
  for i in range(dims):
    for j in range(dims):
      if i != j:
        no_overlapping.append(
                Or(
                X[i] + rot[i] * H[i] + (1 - rot[i]) * W[i] <= X[j],
                Y[i] + rot[i] * W[i] + (1 - rot[i]) * H[i] <= Y[j],
                X[i] - rot[j] * H[j] - (1 - rot[j]) * W[j] >= X[j],
                Y[i] - rot[j] * W[j] - (1 - rot[j]) * H[j] >= Y[j])) 
  all_constraints += no_overlapping

  """
    Solving phase
  """
  solver = Optimize()
  # print(str(all_constraints))
  solver.add(all_constraints)
  # minimizing function
  if time_out != -1:
    solver.set("timeout",time_out*1000)
  solver.minimize(height)
  if solver.check() == sat:
    m = solver.model()
    return int(m.evaluate(height).as_string()), [int(m.evaluate(X[i]).as_string()) for i in range(dims)], \
    [int(m.evaluate(Y[i]).as_string()) for i in range(dims)], \
    [int(m.evaluate(rot[i]).as_string()) for i in range(dims)]
  else:
    print(solver.reason_unknown())
    return None



# Solving
## reading solution time per instance

In [20]:
time_table = "time_table_ext.json"
time_per_instance = []
with open(time_table, "r") as f:
    time_json = json.load(f)

## Getting instances 

In [21]:
path = "../../instances/"
# solve all instances in path
# instances = os.listdir(path)

# solve a certain range of the instances
# instances = [f"ins-{i}.txt" for i in range (10,19)]

# solve a specified instance
instances = [f"ins-{19}.txt"]

# solve those unsolved instances
# instances = []
# for key, value in time_json.items():
#   if value == -1:
#     instances.append(key)


## Solving instances

In [22]:
for i in range(len(instances)):
    # print(path+instance)
    instance = instances[i]
    print(f"Instance {i+1}:")
    width, W, H = read_instance_text(path + instance)
    # print(dims)
    max_height = np.sum([H[i] for i in range(len(H))], dtype=int)
    # print(f"Maximum height reachable: {max_height}")

    start_time = time.time_ns()
    returned_value = vlsi_instance(W, H, len(W), width, max_height)
    if returned_value is not None:
        height, X, Y, rot = returned_value
        end_time = time.time_ns()
        end_time = (end_time - start_time) / 10 ** 9
        print(f"Solution (width, height): {width, height}, time: {round(end_time, 4)}s")
        print(returned_value)
        time_json[instance] = end_time

        # showing solutions
        solutions = dims_sol_unify_rot(W, H, X, Y, rot)
        save_sol_rot("rot_sym_sol/" + instance.replace("ins", "sol"), width, height, W, H, X, Y, rot)
        # print(check_sat_rot((width,height), solutions))
        arr = draw_solution_rot((width, height), solutions)
        show_shape(arr, instance, len(W))
    else:
        time_json[instance] = -1


Instance 1:
canceled


## Saving time table

In [23]:
with open(time_table, "w") as outfile:
    data = json.dumps(time_json, indent=4, sort_keys=True)
    outfile.write(data)

## Performance Analysis

In [24]:
acc = 0
counter = 0
for k,v in time_json.items():
    if v != -1:
        acc +=v
        counter+=1

print(f"Number of successfully solved instances: {counter}")
print(f"Average time per instance: {np.round(acc/counter, decimals=2)}/s")


Number of successfully solved instances: 5
Average time per instance: 110.39/s
